# Structure Learning in Bayesian Networks

In this notebook, we show a few examples of Causal Discovery or Structure Learning in pgmpy. pgmpy currently has the following algorithm for causal discovery:

1. **PC**: Has $3$ variants original, stable, and parallel. PC is a constraint-based algorithm that utilizes Conditional Independence tests to construct the model.
2. **Hill-Climb Search**: Hill-Climb Search is a greedy optimization-based algorithm that makes iterative local changes to the model structure such that it improves the overall score of the model.
3. **Exhaustive Search**: Exhaustive search iterates over all possible network structures on the given variables to find the most optimal one. As it tries to enumerate all possible network structures, it is intractable when the number of variables in the data is large.

The following Conditional Independence Tests are available to use with the PC algorithm:
1. **Pillai**: Test for mixed data based on residualization approach. It is a generalization of Pearsonr to mixed data.
1. **Chi-Square test**: Works only for discrete data.
2. **Pearsonr**: A partial correlation-based test. Works for Gaussian continuous variables.
3. **G-squared**: Works only for discrete data.
4. **Log-likelihood**: Works only for discrete data.

For Hill-Climb and Exhausitive Search the following scoring methods can be used:
1. **BIC Score**
2. **AIC Score**
3. **K2 Score**
4. **BDeU Score**
5. **BDs Score**

## Generate some data

In [1]:
from itertools import combinations

import networkx as nx
from sklearn.metrics import f1_score

from pgmpy.estimators import PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score
from pgmpy.utils import get_example_model
import numpy as np

In [5]:
model = get_example_model("alarm")
samples = model.simulate(int(1e3))
samples.head()

  0%|          | 0/37 [00:00<?, ?it/s]

,CO,ARTCO2,MINVOLSET,FIO2,LVEDVOLUME,VENTLUNG,ANAPHYLAXIS,PRESS,EXPCO2,VENTMACH,...,HRSAT,ERRLOWOUTPUT,HRBP,HR,PULMEMBOLUS,INTUBATION,HYPOVOLEMIA,KINKEDTUBE,PAP,SAO2
0,HIGH,HIGH,NORMAL,NORMAL,LOW,ZERO,FALSE,LOW,LOW,NORMAL,...,NORMAL,FALSE,HIGH,HIGH,FALSE,NORMAL,TRUE,FALSE,NORMAL,LOW
1,HIGH,HIGH,NORMAL,NORMAL,HIGH,ZERO,FALSE,LOW,LOW,NORMAL,...,HIGH,FALSE,HIGH,HIGH,FALSE,NORMAL,TRUE,FALSE,NORMAL,LOW
2,HIGH,HIGH,LOW,NORMAL,NORMAL,ZERO,FALSE,HIGH,LOW,LOW,...,HIGH,FALSE,HIGH,HIGH,FALSE,NORMAL,FALSE,FALSE,NORMAL,LOW
3,LOW,LOW,HIGH,NORMAL,LOW,LOW,FALSE,LOW,LOW,HIGH,...,HIGH,TRUE,LOW,NORMAL,FALSE,NORMAL,TRUE,FALSE,NORMAL,HIGH
4,HIGH,HIGH,NORMAL,NORMAL,NORMAL,ZERO,FALSE,HIGH,LOW,NORMAL,...,HIGH,FALSE,HIGH,HIGH,FALSE,NORMAL,FALSE,FALSE,NORMAL,LOW


In [6]:
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_array(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_array(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)

## Learn the model structure using PC

In [ ]:
est = PC(data=samples)
estimated_model = est.estimate(variant="stable", max_cond_vars=4)
get_f1_score(estimated_model, model)

  0%|          | 0/4 [00:00<?, ?it/s]

/home/ankur/pgmpy/examples/pgmpy/estimators/CITests.py:542: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_
/home/ankur/pgmpy/examples/pgmpy/estimators/CITests.py:542: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby([X, Y]).size().unstack(Y, fill_value=0), lambda_=lambda_
/home/ankur/pgmpy/examples/pgmpy/estimators/CITests.py:542: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future de

In [ ]:
est = PC(data=samples)
estimated_model = est.estimate(variant="orig", max_cond_vars=4)
get_f1_score(estimated_model, model)

## Learn the model structure using Hill-Climb Search

In [ ]:
scoring_method = K2Score(data=samples)
est = HillClimbSearch(data=samples)
estimated_model = est.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)
)
get_f1_score(estimated_model, model)